In [1]:
import sys
from datetime import datetime
from os.path import join
from warnings import warn
from itertools import chain

import numpy as np
import pandas as pd
import scipy as sp

from utils import *

In [2]:
dir_arg = sys.argv[1]
if dir_arg == '-f':
    file_dir = join('..', 'dataset', '11')
else:
    file_dir = join('..', 'dataset',  dir_arg)

In [106]:
train_df = pd.read_pickle(join(file_dir, 'base_feauture.pkl'))

sample = pd.read_pickle(join(file_dir, 'uid.pkl'))

now_date = train_df.orderdate.max().date()
print(datetime.now(), now_date)

uid_shape, hotelid_shape, basicroomid_shape, roomid_shape = print_shape(
    train_df, ['uid', 'hotelid', 'basicroomid', 'roomid'])

2017-07-30 23:12:32.529222 2017-06-11
2017-07-30 23:12:32.529405
--------------------
uid uniuqe shape 3552
hotelid uniuqe shape 3194
basicroomid uniuqe shape 19542
roomid uniuqe shape 121802
--------------------


In [4]:
feature_path = join(file_dir, 'user_feature.pkl')
print(datetime.now(), 'begin', feature_path)

2017-07-30 22:32:23.710733 begin ../dataset/11/user_feature.pkl


In [67]:
user_oreder_type = [x for x in train_df.columns if x.startswith('ordertype')]
user_orderbehavior = [x for x in train_df.columns if x.startswith('orderbehavior')]
user_lastord = [x for x in train_df.columns if x.endswith('lastord')]
user_feature_cols = [x for x in train_df.columns if x.startswith('user')]

## 添加基本特征

In [6]:
(now_date - train_df.orderdate_lastord).dt.days.max()

456.0

In [7]:
train_df.orderdate_lastord.min()

Timestamp('2016-03-12 00:00:00')

In [37]:
(now_date - train_df.orderdate_lastord).dt.days.isnull().sum()

11080

In [15]:
sample = extract_feature_count('uid', 'hotel_roomid', train_df, sample)

In [46]:
def extract_user_feature_is_equal(t, train_df, sample):
    name = '{}_is_equal'.format(t)
    lastord_name = t + '_lastord'
    train_df[name] = np.nan
    train_df.loc[(train_df[lastord_name] == train_df[t]), name] = 1
    sample = extract_feature_count('uid', name, train_df, sample)
    sample = press_date(sample, ['uid' + '__' + name + '_count'])
    return sample

In [49]:
for i in range(2, 9):
    t = 'roomservice_%d' % i
    if i != 7:
        sample = extract_user_feature_is_equal(t, train_df, sample)

In [57]:
for i in range(2, 5):
    t = 'roomtag_%d' % i
    sample = extract_user_feature_is_equal(t, train_df, sample)

In [59]:
for c in ['rank', 'star']:
    sample = extract_user_feature_is_equal(c, train_df, sample)

In [68]:
user_cols = list(chain(user_oreder_type, user_orderbehavior, user_feature_cols))

In [99]:
add_cols = ['hotel_minprice_lastord', 'basic_minprice_lastord', 'star_lastord'] + user_cols

In [107]:
not_use = [
    'ordertype_1_ratio', 'ordertype_2_ratio', 'ordertype_4_ratio',
    'ordertype_5_ratio', 'ordertype_7_ratio', 'ordertype_9_ratio',
    'ordertype_11_ratio', 'orderbehavior_1_ratio',
    'orderbehavior_3_ratio_1week', 'orderbehavior_4_ratio_1week',
    'orderbehavior_3_ratio_3month', 'orderbehavior_4_ratio_3month',
    'orderbehavior_5_ratio_3month', 'orderbehavior_6_ratio', 'orderbehavior_8',
    'user_confirmtime', 'user_avgadvanceddate', 'user_avgroomnum',
    'user_avgpromotion', 'user_avgroomarea', 'user_roomservice_4_0ratio',
    'user_roomservice_4_3ratio', 'user_roomservice_4_4ratio',
    'user_roomservice_4_5ratio', 'user_roomservice_3_123ratio',
    'user_roomservice_6_2ratio', 'user_roomservice_6_1ratio',
    'user_roomservice_6_0ratio', 'user_roomservice_5_1ratio',
    'user_roomservice_2_1ratio', 'user_roomservice_8_345ratio',
    'user_ordnum_1week', 'user_roomservice_7_1ratio_1week',
    'user_roomservice_7_0ratio_1week', 'user_roomservice_4_5ratio_1week',
    'user_roomservice_4_3ratio_1week', 'user_roomservice_4_0ratio_1week',
    'user_ordnum_1month', 'user_roomservice_3_123ratio_1month',
    'user_roomservice_7_1ratio_1month', 'user_roomservice_7_0ratio_1month',
    'user_roomservice_4_5ratio_1month', 'user_roomservice_4_3ratio_1month',
    'user_roomservice_4_0ratio_1month', 'user_ordnum_3month',
    'user_roomservice_3_123ratio_3month', 'user_roomservice_7_1ratio_3month',
    'user_roomservice_7_0ratio_3month', 'user_roomservice_4_5ratio_3month',
    'user_roomservice_4_3ratio_3month', 'user_roomservice_4_0ratio_3month',
    'ordertype_3_ratio', 'user_roomservice_4_1ratio'
]
not_use = []

In [108]:
for col in add_cols:
    if col not in not_use:
        sample = add_column(train_df, sample, 'uid', col)

In [109]:
get_corr(train_df, sample, 'uid')

,orderlabel,uid_hotel_minprice_lastord,uid_basic_minprice_lastord,uid_star_lastord,uid_ordertype_6_ratio,uid_ordertype_8_ratio,uid_ordertype_10_ratio,uid_orderbehavior_2_ratio,uid_orderbehavior_5_ratio_1week,uid_orderbehavior_7_ratio,...,uid_user_minprice_1month,uid_user_maxprice_1month,uid_user_roomservice_4_4ratio_1month,uid_user_roomservice_4_2ratio_1month,uid_user_avgprice_3month,uid_user_medprice_3month,uid_user_minprice_3month,uid_user_maxprice_3month,uid_user_roomservice_4_4ratio_3month,uid_user_roomservice_4_2ratio_3month
orderlabel,1.000000,-0.027461,-0.020445,-0.035447,0.010275,0.014661,0.010698,0.011683,-0.011700,0.011365,...,-0.029753,-0.024625,-0.014673,0.014063,-0.034039,-0.034202,-0.028349,-0.025748,-0.012091,0.014541
uid_hotel_minprice_lastord,-0.027461,1.000000,0.772945,0.571928,-0.033196,-0.214359,-0.044067,-0.120612,0.158647,-0.125571,...,0.671298,0.630284,0.326356,-0.189293,0.713103,0.693761,0.532592,0.573151,0.226352,-0.158326
uid_basic_minprice_lastord,-0.020445,0.772945,1.000000,0.430442,-0.031105,-0.180300,-0.038022,-0.085796,0.165316,-0.094069,...,0.470150,0.792085,0.202508,-0.134389,0.693805,0.609122,0.434109,0.696117,0.122719,-0.112580
uid_star_lastord,-0.035447,0.571928,0.430442,1.000000,-0.082509,-0.247461,-0.056786,-0.156405,0.059683,-0.188180,...,0.496231,0.435995,0.235794,-0.213287,0.508652,0.500669,0.401590,0.413406,0.152475,-0.211841
uid_ordertype_6_ratio,0.010275,-0.033196,-0.031105,-0.082509,1.000000,0.154829,0.179453,0.271675,0.043387,0.202086,...,-0.022764,-0.058936,-0.006376,0.097220,-0.063177,-0.053691,0.008488,-0.085930,0.018201,0.145894
uid_ordertype_8_ratio,0.014661,-0.214359,-0.180300,-0.247461,0.154829,1.000000,0.078451,0.221567,-0.108731,0.215153,...,-0.188727,-0.213998,-0.051393,0.118899,-0.253284,-0.232948,-0.129347,-0.259362,-0.042830,0.194363
uid_ordertype_10_ratio,0.010698,-0.044067,-0.038022,-0.056786,0.179453,0.078451,1.000000,0.179792,0.046650,0.071451,...,-0.039561,-0.049692,-0.029110,0.051202,-0.068002,-0.058607,-0.029315,-0.074918,-0.024621,0.038592
uid_orderbehavior_2_ratio,0.011683,-0.120612,-0.085796,-0.156405,0.271675,0.221567,0.179792,1.000000,0.002431,0.322033,...,-0.108654,-0.056495,-0.044619,0.086841,-0.113678,-0.113168,-0.071277,-0.084963,-0.038433,0.132975
uid_orderbehavior_5_ratio_1week,-0.011700,0.158647,0.165316,0.059683,0.043387,-0.108731,0.046650,0.002431,1.000000,-0.001792,...,0.263029,0.193204,0.042777,0.089158,0.236430,0.256635,0.221046,0.132163,0.034868,0.044858
uid_orderbehavior_7_ratio,0.011365,-0.125571,-0.094069,-0.188180,0.202086,0.215153,0.071451,0.322033,-0.001792,1.000000,...,-0.071613,-0.109877,-0.049352,0.149668,-0.103473,-0.095280,-0.015419,-0.120058,-0.022736,0.199398


In [74]:
# corr = _

# cor = abs(corr['orderlabel'])

# [x for x in cor.loc[np.isnan(cor)].index]

# [x[4:] for x in cor.loc[cor<0.01].index]  + [x[4:] for x in cor.loc[np.isnan(cor)].index]

## 历史价格统计特征 

In [9]:
name_fmt = '{}_diff_{}'.format('uid', '{}')

price_diff_name = name_fmt.format('price_last_lastord')
hotel_minprice_diff_name = name_fmt.format('hotel_minprice_lastord')
basic_minprice_diff_name = name_fmt.format('basic_minprice_lastord')

In [10]:
train_df[price_diff_name] = train_df['price_deduct'] - train_df['price_last_lastord']
train_df[hotel_minprice_diff_name] = train_df['price_deduct'] - train_df['hotel_minprice_lastord']
train_df[basic_minprice_diff_name] = train_df['price_deduct'] - train_df['basic_minprice_lastord']

In [ ]:
price_describe = ['mean', '75%']

In [11]:
sample = extract_value_describe_feature('uid', price_diff_name, train_df, sample, price_describe)
sample = extract_value_describe_feature('uid', hotel_minprice_diff_name, train_df, sample, price_describe)
sample = extract_value_describe_feature('uid', basic_minprice_diff_name, train_df, sample, price_describe)

In [16]:
# get_corr(train_df, sample, 'uid')

,orderlabel,uid__uid_diff_price_last_lastord_count,uid__uid_diff_price_last_lastord_mean,uid__uid_diff_price_last_lastord_std,uid__uid_diff_price_last_lastord_min,uid__uid_diff_price_last_lastord_25,uid__uid_diff_price_last_lastord_50,uid__uid_diff_price_last_lastord_75,uid__uid_diff_price_last_lastord_max,uid__uid_diff_hotel_minprice_lastord_count,...,uid__uid_diff_hotel_minprice_lastord_max,uid__uid_diff_basic_minprice_lastord_count,uid__uid_diff_basic_minprice_lastord_mean,uid__uid_diff_basic_minprice_lastord_std,uid__uid_diff_basic_minprice_lastord_min,uid__uid_diff_basic_minprice_lastord_25,uid__uid_diff_basic_minprice_lastord_50,uid__uid_diff_basic_minprice_lastord_75,uid__uid_diff_basic_minprice_lastord_max,uid__hotel_roomid_count
orderlabel,1.000000,-0.082468,-0.027297,-0.019170,-0.006431,-0.017837,-0.025349,-0.032527,-0.020614,-0.082468,...,-0.020758,-0.082468,-0.027825,-0.019170,-0.007673,-0.018998,-0.026308,-0.033209,-0.020662,-0.092254
uid__uid_diff_price_last_lastord_count,-0.082468,1.000000,0.287712,0.183210,0.108050,0.225623,0.303479,0.358626,0.210281,1.000000,...,0.211220,1.000000,0.292979,0.183210,0.120997,0.237552,0.313198,0.365457,0.210759,0.910141
uid__uid_diff_price_last_lastord_mean,-0.027297,0.287712,1.000000,0.796070,0.636920,0.742495,0.810900,0.856718,0.783407,0.287712,...,0.774295,0.287712,0.999195,0.796070,0.645171,0.747981,0.813461,0.857065,0.783188,0.287712
uid__uid_diff_price_last_lastord_std,-0.019170,0.183210,0.796070,1.000000,0.132056,0.234659,0.325356,0.430358,0.977654,0.183210,...,0.978502,0.183210,0.798789,1.000000,0.141157,0.242947,0.331959,0.434760,0.977695,0.183210
uid__uid_diff_price_last_lastord_min,-0.006431,0.108050,0.636920,0.132056,1.000000,0.969501,0.900895,0.754034,0.179880,0.108050,...,0.158218,0.108050,0.628860,0.132056,0.996609,0.962180,0.891422,0.745002,0.179105,0.108050
uid__uid_diff_price_last_lastord_25,-0.017837,0.225623,0.742495,0.234659,0.969501,1.000000,0.973238,0.866761,0.279373,0.225623,...,0.260349,0.225623,0.736647,0.234659,0.970605,0.997232,0.967678,0.860579,0.278794,0.225623
uid__uid_diff_price_last_lastord_50,-0.025349,0.303479,0.810900,0.325356,0.900895,0.973238,1.000000,0.940006,0.364421,0.303479,...,0.348563,0.303479,0.807122,0.325356,0.905658,0.974302,0.997945,0.936450,0.364025,0.303479
uid__uid_diff_price_last_lastord_75,-0.032527,0.358626,0.856718,0.430358,0.754034,0.866761,0.940006,1.000000,0.450263,0.358626,...,0.439372,0.358626,0.854808,0.430358,0.761046,0.870707,0.940870,0.998792,0.450026,0.358626
uid__uid_diff_price_last_lastord_max,-0.020614,0.210281,0.783407,0.977654,0.179880,0.279373,0.364421,0.450263,1.000000,0.210281,...,0.999525,0.210281,0.785721,0.977654,0.188675,0.287172,0.370451,0.454149,0.999992,0.210281
uid__uid_diff_hotel_minprice_lastord_count,-0.082468,1.000000,0.287712,0.183210,0.108050,0.225623,0.303479,0.358626,0.210281,1.000000,...,0.211220,1.000000,0.292979,0.183210,0.120997,0.237552,0.313198,0.365457,0.210759,0.910141


### 历史订单间隔统计特征 

In [38]:
span_name, t = '{}_ordspan'.format('uid'), 'uid'

In [43]:
# train_df[span_name] = (now_date - train_df.orderdate_lastord).dt.days

# sample = extract_value_describe_feature(t, span_name, train_df, sample, ['max', 'min', 'mean'])

In [42]:
# get_corr(train_df, sample, 'uid')

,orderlabel,uid__uid_diff_price_last_lastord_count,uid__uid_diff_price_last_lastord_mean,uid__uid_diff_price_last_lastord_std,uid__uid_diff_price_last_lastord_min,uid__uid_diff_price_last_lastord_25,uid__uid_diff_price_last_lastord_50,uid__uid_diff_price_last_lastord_75,uid__uid_diff_price_last_lastord_max,uid__uid_diff_hotel_minprice_lastord_count,...,uid__uid_diff_basic_minprice_lastord_mean,uid__uid_diff_basic_minprice_lastord_std,uid__uid_diff_basic_minprice_lastord_min,uid__uid_diff_basic_minprice_lastord_25,uid__uid_diff_basic_minprice_lastord_50,uid__uid_diff_basic_minprice_lastord_75,uid__uid_diff_basic_minprice_lastord_max,uid__uid_ordspan_max,uid__uid_ordspan_min,uid__uid_ordspan_mean
orderlabel,1.000000,-0.082468,-0.027297,-0.019170,-0.006431,-0.017837,-0.025349,-0.032527,-0.020614,-0.082468,...,-0.027825,-0.019170,-0.007673,-0.018998,-0.026308,-0.033209,-0.020662,0.003112,0.003112,0.003112
uid__uid_diff_price_last_lastord_count,-0.082468,1.000000,0.287712,0.183210,0.108050,0.225623,0.303479,0.358626,0.210281,1.000000,...,0.292979,0.183210,0.120997,0.237552,0.313198,0.365457,0.210759,0.016251,0.016251,0.016251
uid__uid_diff_price_last_lastord_mean,-0.027297,0.287712,1.000000,0.796070,0.636920,0.742495,0.810900,0.856718,0.783407,0.287712,...,0.999195,0.796070,0.645171,0.747981,0.813461,0.857065,0.783188,0.013835,0.013835,0.013835
uid__uid_diff_price_last_lastord_std,-0.019170,0.183210,0.796070,1.000000,0.132056,0.234659,0.325356,0.430358,0.977654,0.183210,...,0.798789,1.000000,0.141157,0.242947,0.331959,0.434760,0.977695,0.012476,0.012476,0.012476
uid__uid_diff_price_last_lastord_min,-0.006431,0.108050,0.636920,0.132056,1.000000,0.969501,0.900895,0.754034,0.179880,0.108050,...,0.628860,0.132056,0.996609,0.962180,0.891422,0.745002,0.179105,0.014939,0.014939,0.014939
uid__uid_diff_price_last_lastord_25,-0.017837,0.225623,0.742495,0.234659,0.969501,1.000000,0.973238,0.866761,0.279373,0.225623,...,0.736647,0.234659,0.970605,0.997232,0.967678,0.860579,0.278794,0.022829,0.022829,0.022829
uid__uid_diff_price_last_lastord_50,-0.025349,0.303479,0.810900,0.325356,0.900895,0.973238,1.000000,0.940006,0.364421,0.303479,...,0.807122,0.325356,0.905658,0.974302,0.997945,0.936450,0.364025,0.026856,0.026856,0.026856
uid__uid_diff_price_last_lastord_75,-0.032527,0.358626,0.856718,0.430358,0.754034,0.866761,0.940006,1.000000,0.450263,0.358626,...,0.854808,0.430358,0.761046,0.870707,0.940870,0.998792,0.450026,0.018517,0.018517,0.018517
uid__uid_diff_price_last_lastord_max,-0.020614,0.210281,0.783407,0.977654,0.179880,0.279373,0.364421,0.450263,1.000000,0.210281,...,0.785721,0.977654,0.188675,0.287172,0.370451,0.454149,0.999992,0.019975,0.019975,0.019975
uid__uid_diff_hotel_minprice_lastord_count,-0.082468,1.000000,0.287712,0.183210,0.108050,0.225623,0.303479,0.358626,0.210281,1.000000,...,0.292979,0.183210,0.120997,0.237552,0.313198,0.365457,0.210759,0.016251,0.016251,0.016251


In [28]:
train_df.shape

(130801, 154)

In [ ]:
sample.to_pickle(feature_path)

print(datetime.now(), 'save to', feature_path)